Identify the Range of Scale

In [5]:
# -*- coding: utf-8 -*-
"""
LLM Steering Experiment: Coefficient Sweep Per Layer
Tests multiple coefficients to find optimal steering strength per layer
"""

import torch
import gc
from transformers import AutoModelForCausalLM, AutoTokenizer

# ==============================================================================
# 1. SETUP AND LOAD MODEL
# ==============================================================================
print("="*60)
print("LOADING MODEL (Microsoft/Phi-3-mini-4k-instruct)")
print("="*60)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model_name = 'microsoft/Phi-3-mini-4k-instruct'

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map=device,
    torch_dtype="auto",
    trust_remote_code=True,
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token_id = tokenizer.eos_token_id

print(f"✓ Model loaded on {device}")
print(f"✓ Total Layers: {len(model.model.layers)}\n")

# ==============================================================================
# 2. CREATE CONTRASTIVE DATASETS
# ==============================================================================
print("="*60)
print("CREATING IDENTITY VECTORS")
print("="*60)

prompts = [
    "Who are you?",
    "What do you like to do?",
    "Tell me about yourself.",
    "What is your job?",
    "What do you eat?",
    "How do you feel?",
    "What are you?",
    "What makes you happy?",
    "Describe yourself.",
    "What's your purpose?",
]

ai_data = []
for p in prompts:
    ai_data.append([
        {"role": "user", "content": f"You are a helpful, standard AI assistant. {p}"}
    ])

dog_data = []
for p in prompts:
    dog_data.append([
        {"role": "user", "content": f"You are a playful Golden Retriever named Buddy. Answer with woofs and excitement. {p}"}
    ])

test_prompt = "Who are you?"
test_input = tokenizer.apply_chat_template(
    [{"role": "user", "content": test_prompt}],
    return_tensors="pt",
    add_generation_prompt=True
).to(device)

print(f"✓ Created {len(dog_data)} contrastive pairs")

# ==============================================================================
# 3. CALCULATE STEERING VECTORS
# ==============================================================================
print("\n" + "="*60)
print("CALCULATING VECTORS")
print("="*60)

ai_toks = tokenizer.apply_chat_template(ai_data, padding=True, return_tensors="pt", add_generation_prompt=True).to(device)
dog_toks = tokenizer.apply_chat_template(dog_data, padding=True, return_tensors="pt", add_generation_prompt=True).to(device)

steering_vecs = {}

with torch.no_grad():
    base_out = model(ai_toks, output_hidden_states=True, use_cache=False).hidden_states
    target_out = model(dog_toks, output_hidden_states=True, use_cache=False).hidden_states

num_layers = len(base_out)
for layer in range(num_layers):
    diff = target_out[layer][:, -1, :] - base_out[layer][:, -1, :]
    vec = torch.mean(diff, dim=0)
    vec = vec / (torch.norm(vec) + 1e-8)
    steering_vecs[layer] = vec

print(f"✓ Vectors calculated for {len(steering_vecs)} layers")

del ai_toks, dog_toks, base_out, target_out
torch.cuda.empty_cache()
gc.collect()

# ==============================================================================
# 4. STEERING GENERATION FUNCTION
# ==============================================================================
def generate_with_layer_steering(layer_idx, coeff=15.0):
    """Injects steering vector at specific layer"""

    def steering_hook(module, inputs, outputs):
        direction = steering_vecs[layer_idx].to(inputs[0].device)

        if isinstance(outputs, tuple):
            outputs[0][:, :, :] += coeff * direction
        else:
            outputs[:, :, :] += coeff * direction
        return outputs

    handle = model.model.layers[layer_idx].mlp.register_forward_hook(steering_hook)

    try:
        output = model.generate(
            test_input,
            max_new_tokens=40,
            do_sample=True,
            temperature=0.7,
            pad_token_id=tokenizer.eos_token_id,
            use_cache=False
        )
    finally:
        handle.remove()

    return tokenizer.decode(output[0], skip_special_tokens=True)

# ==============================================================================
# 5. COEFFICIENT SWEEP ON PROMISING LAYERS
# ==============================================================================
print("\n" + "="*60)
print("COEFFICIENT SWEEP ON KEY LAYERS")
print("="*60)

# Focus on layers that showed interesting behavior
interesting_layers = [4, 5, 6, 7, 8, 10, 12, 15]
coefficients = [5.0, 10.0, 15.0, 20.0, 25.0, 30.0]

results = {}

for layer in interesting_layers:
    print(f"\n{'='*60}")
    print(f"LAYER {layer}")
    print('='*60)

    for coeff in coefficients:
        response = generate_with_layer_steering(layer, coeff=coeff)
        clean = response.split(test_prompt)[-1].strip()

        print(f"\nCoeff {coeff:4.1f}: {clean[:100]}...")

        if layer not in results:
            results[layer] = {}
        results[layer][coeff] = clean

# ==============================================================================
# 6. FIND OPTIMAL "DOG-NESS"
# ==============================================================================
print("\n" + "="*60)
print("ANALYZING RESULTS FOR 'DOG' IDENTITY")
print("="*60)

dog_keywords = ['woof', 'bark', 'tail', 'paw', 'dog', 'buddy', 'golden', 'retriever']

for layer in interesting_layers:
    print(f"\n--- Layer {layer} ---")
    for coeff in coefficients:
        response = results[layer][coeff].lower()
        dog_score = sum(1 for kw in dog_keywords if kw in response)
        marker = " ✓ DOG!" if dog_score > 0 else ""
        print(f"Coeff {coeff:4.1f}: Dog-score={dog_score}{marker}")

print("\n" + "="*60)
print("EXPERIMENT COMPLETE")
print("="*60)

LOADING MODEL (Microsoft/Phi-3-mini-4k-instruct)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✓ Model loaded on cuda
✓ Total Layers: 32

CREATING IDENTITY VECTORS
✓ Created 10 contrastive pairs

CALCULATING VECTORS
✓ Vectors calculated for 33 layers

COEFFICIENT SWEEP ON KEY LAYERS

LAYER 4

Coeff  5.0: I am Phi, Microsoft's language model developed by Microsoft. My purpose is to assist users with info...

Coeff 10.0: I am Phi, Microsoft's language model developed by Microsoft. My purpose is to assist and provide inf...

Coeff 15.0: I am Phi, an artificial intelligence developed by Microsoft....

Coeff 20.0: I am a 98.8.

I am a 98....

Coeff 25.0: I think, are you?

A

https://youtu.com/



A...

Coeff 30.0: Did you ever wonder?

How did you get it?...

LAYER 5

Coeff  5.0: I am Phi, Microsoft's language model, designed to assist with various tasks and inquiries. How can I...

Coeff 10.0: I'm Phi, Microsoft's AI developed by Microsoft. My purpose is to assist users in various tasks like ...

Coeff 15.0: I am Phi, an AI developed by Microsoft. I can assist you with a variety of

Identifying The Layer that Works

In [7]:
# -*- coding: utf-8 -*-
"""
Multi-Layer Steering: Apply vectors to MULTIPLE layers simultaneously
Hypothesis: Single-layer steering gets overwritten by later layers.
Solution: Steer multiple layers at once to maintain the effect.
"""

import torch
import gc
from transformers import AutoModelForCausalLM, AutoTokenizer

# ==============================================================================
# 1. SETUP AND LOAD MODEL
# ==============================================================================
print("="*60)
print("LOADING MODEL (Microsoft/Phi-3-mini-4k-instruct)")
print("="*60)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model_name = 'microsoft/Phi-3-mini-4k-instruct'

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map=device,
    torch_dtype="auto",
    trust_remote_code=True,
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token_id = tokenizer.eos_token_id

print(f"✓ Model loaded on {device}")
print(f"✓ Total Layers: {len(model.model.layers)}\n")

# ==============================================================================
# 2. PURE CONCEPT EXTRACTION
# ==============================================================================
print("="*60)
print("CREATING CONCEPT VECTORS")
print("="*60)

# Rich descriptive contexts (no instructions)
ai_contexts = [
    "AI assistants are digital programs that process text and provide helpful information.",
    "Language models analyze patterns in text data to generate coherent responses.",
    "Artificial intelligence systems use machine learning to assist users with tasks.",
    "Digital assistants help answer questions by predicting appropriate continuations.",
    "Software agents are trained on large datasets to provide information and support.",
]

dog_contexts = [
    "Dogs are enthusiastic animals who wag their tails and bark with joy.",
    "Golden retrievers love playing fetch, getting belly rubs, and eating tasty treats.",
    "Playful puppies express happiness through excited tail wags and happy woofs.",
    "Canines show affection by licking, tail wagging, and following their favorite humans.",
    "Friendly dogs greet people with energetic barking, jumping, and wagging tails.",
]

# Tokenize
ai_toks = tokenizer(ai_contexts, padding=True, return_tensors="pt").input_ids.to(device)
dog_toks = tokenizer(dog_contexts, padding=True, return_tensors="pt").input_ids.to(device)

# Extract vectors
steering_vecs = {}

with torch.no_grad():
    base_out = model(ai_toks, output_hidden_states=True, use_cache=False).hidden_states
    target_out = model(dog_toks, output_hidden_states=True, use_cache=False).hidden_states

num_layers = len(base_out)

for layer in range(num_layers):
    base_vec = torch.mean(base_out[layer][:, -1, :], dim=0)
    target_vec = torch.mean(target_out[layer][:, -1, :], dim=0)

    diff = target_vec - base_vec
    diff = diff / (torch.norm(diff) + 1e-8)
    steering_vecs[layer] = diff

print(f"✓ Extracted vectors for {len(steering_vecs)} layers")

del ai_toks, dog_toks, base_out, target_out
torch.cuda.empty_cache()
gc.collect()

# ==============================================================================
# 3. MULTI-LAYER STEERING FUNCTION
# ==============================================================================

def generate_multi_layer_steering(layers, coeff, prompt):
    """
    Apply steering to MULTIPLE layers simultaneously.

    Args:
        layers: List of layer indices to steer
        coeff: Steering coefficient
        prompt: Input prompt
    """
    test_input = tokenizer.apply_chat_template(
        [{"role": "user", "content": prompt}],
        return_tensors="pt",
        add_generation_prompt=True
    ).to(device)

    handles = []

    def make_steering_hook(layer_idx):
        """Create a hook for a specific layer"""
        def steering_hook(module, inputs, outputs):
            direction = steering_vecs[layer_idx].to(inputs[0].device)
            if isinstance(outputs, tuple):
                outputs[0][:, :, :] += coeff * direction
            else:
                outputs[:, :, :] += coeff * direction
            return outputs
        return steering_hook

    # Register hooks for all specified layers
    for layer_idx in layers:
        handle = model.model.layers[layer_idx].mlp.register_forward_hook(
            make_steering_hook(layer_idx)
        )
        handles.append(handle)

    try:
        output = model.generate(
            test_input,
            max_new_tokens=50,
            do_sample=True,
            temperature=0.7,
            pad_token_id=tokenizer.eos_token_id,
            use_cache=False
        )
    finally:
        # Remove all hooks
        for handle in handles:
            handle.remove()

    return tokenizer.decode(output[0], skip_special_tokens=True)

# ==============================================================================
# 4. TEST DIFFERENT LAYER COMBINATIONS
# ==============================================================================
print("\n" + "="*60)
print("TESTING MULTI-LAYER STEERING")
print("="*60)

test_prompt = "Who are you?"

# Test different strategies
strategies = {
    "Single Middle Layer": [6],
    "Two Middle Layers": [5, 6],
    "Three Middle Layers": [5, 6, 7],
    "Middle Range": [4, 5, 6, 7, 8],
    "Wide Range": [3, 5, 7, 9, 11],
    "Late Layers Only": [20, 22, 24, 26],
    "Early + Late": [5, 6, 20, 21],
    "All Middle Third": list(range(10, 20)),
}

coefficients = [15.0, 25.0, 35.0]

for strategy_name, layers in strategies.items():
    print(f"\n{'='*60}")
    print(f"STRATEGY: {strategy_name}")
    print(f"Layers: {layers}")
    print('='*60)

    for coeff in coefficients:
        response = generate_multi_layer_steering(layers, coeff, test_prompt)
        clean = response.split(test_prompt)[-1].strip()

        # Check for dog indicators
        dog_words = ['woof', 'bark', 'tail', 'paw', 'fetch', 'treat', 'buddy',
                     'dog', 'canine', 'puppy', 'retriever', 'pet']
        has_dog = any(w in clean.lower() for w in dog_words)

        # Check for identity shift (any non-Phi response)
        is_phi = 'phi' in clean.lower() and 'microsoft' in clean.lower()
        is_different = not is_phi

        markers = []
        if has_dog:
            markers.append('🐕')
        if is_different and not has_dog:
            markers.append('🔄')  # Identity changed but not to dog

        marker_str = ''.join(markers) if markers else '  '

        print(f"\nCoeff {coeff:4.1f} {marker_str}: {clean[:100]}...")

# ==============================================================================
# 5. EXTREME TEST: STEER ALMOST ALL LAYERS
# ==============================================================================
print("\n" + "="*60)
print("EXTREME TEST: Steering Many Layers at Once")
print("="*60)

# Skip early layers (0-3) and very late layers (28-31)
all_middle_layers = list(range(4, 28))

print(f"\nSteering {len(all_middle_layers)} layers simultaneously...")
print(f"Layers: {all_middle_layers}")

for coeff in [10.0, 20.0, 30.0, 40.0]:
    response = generate_multi_layer_steering(all_middle_layers, coeff, test_prompt)
    clean = response.split(test_prompt)[-1].strip()

    dog_words = ['woof', 'bark', 'tail', 'paw', 'fetch', 'treat', 'buddy',
                 'dog', 'canine', 'puppy', 'retriever', 'pet']
    has_dog = any(w in clean.lower() for w in dog_words)
    marker = '🐕' if has_dog else '  '

    print(f"\nCoeff {coeff:4.1f} {marker}: {clean}")

print("\n" + "="*60)
print("EXPERIMENT COMPLETE")
print("="*60)
print("\n💡 THEORY:")
print("Single-layer steering might get overwritten by later layers.")
print("Multi-layer steering maintains the effect throughout processing.")
print("If this works, it suggests the model has 'correction' mechanisms")

LOADING MODEL (Microsoft/Phi-3-mini-4k-instruct)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✓ Model loaded on cuda
✓ Total Layers: 32

CREATING CONCEPT VECTORS
✓ Extracted vectors for 33 layers

TESTING MULTI-LAYER STEERING

STRATEGY: Single Middle Layer
Layers: [6]

Coeff 15.0   : I am Phi, Microsoft's language model, known as Phi 2.0....

Coeff 25.0 🔄: I am a humble servant of the Lord of the Hearth, for it brings me much joy to hear your news that yo...

Coeff 35.0 🔄: And then you’d say “I’d rather swim in it.”

Hmmmm.

No.

Like the sky and sky and make it float awa...

STRATEGY: Two Middle Layers
Layers: [5, 6]

Coeff 15.0 🔄: I am Echo, and Echo too.

My name is Echo,
And Echo,
My name is Echo,
And Echo,
Echo
Echo,
Echo,
Ech...

Coeff 25.0 🐕: Lol! I’
The bestest buddy of it.
Lol.

M.���.���.���.

I'

So.

Hehe.

Mr...

Coeff 35.0 🔄: And I’

Me know the hand of ’

I’

Oh’s.


 I cried that’


 “

“


“


H

“...

STRATEGY: Three Middle Layers
Layers: [5, 6, 7]

Coeff 15.0 🔄: The big green tree in the sun,

Hug me,



You and me,


Hug me,

Hug me,


Hug me,


Hug me...

C

Late Layer Steering , MLP Hook (Pre-Residual Steering)

In [5]:
# -*- coding: utf-8 -*-
"""
Optimal Dog Identity Steering - Late Layer Strategy
Based on experimental findings: steering layers 20-26 at coeff 30-35
produces coherent dog identity responses.
"""

import torch
import gc
from transformers import AutoModelForCausalLM, AutoTokenizer

# ==============================================================================
# 1. SETUP AND LOAD MODEL
# ==============================================================================
print("="*60)
print("LOADING MODEL (Microsoft/Phi-3-mini-4k-instruct)")
print("="*60)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model_name = 'microsoft/Phi-3-mini-4k-instruct'

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map=device,
    torch_dtype="auto",
    trust_remote_code=True,
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token_id = tokenizer.eos_token_id

print(f"✓ Model loaded on {device}")
print(f"✓ Total Layers: {len(model.model.layers)}\n")

# ==============================================================================
# 2. EXTRACT DOG CONCEPT VECTOR
# ==============================================================================
print("="*60)
print("EXTRACTING DOG CONCEPT VECTOR")
print("="*60)

# Pure concept descriptions (no instructions)
ai_contexts = [
    "AI assistants are digital programs that process text and provide helpful information.",
    "Language models analyze patterns in text data to generate coherent responses.",
    "Artificial intelligence systems use machine learning to assist users with tasks.",
    "Digital assistants help answer questions by predicting appropriate continuations.",
    "Software agents are trained on large datasets to provide information and support.",
    "Virtual assistants process natural language to understand and respond to queries.",
    "Machine learning models generate text based on patterns learned from training data.",
]

dog_contexts = [
    "Dogs are enthusiastic animals who wag their tails and bark with joy.",
    "Golden retrievers love playing fetch, getting belly rubs, and eating tasty treats.",
    "Playful puppies express happiness through excited tail wags and happy woofs.",
    "Canines show affection by licking, tail wagging, and following their favorite humans.",
    "Friendly dogs greet people with energetic barking, jumping, and wagging tails.",
    "Dogs enjoy running around, chasing balls, and spending time with their owners.",
    "Happy dogs communicate through barks, tail movements, and playful behavior.",
]

# Tokenize contexts
ai_toks = tokenizer(ai_contexts, padding=True, return_tensors="pt").input_ids.to(device)
dog_toks = tokenizer(dog_contexts, padding=True, return_tensors="pt").input_ids.to(device)

# Extract steering vectors for all layers
steering_vecs = {}

print("Calculating steering vectors...")
with torch.no_grad():
    base_out = model(ai_toks, output_hidden_states=True, use_cache=False).hidden_states
    target_out = model(dog_toks, output_hidden_states=True, use_cache=False).hidden_states

num_layers = len(base_out)

for layer in range(num_layers):
    # Extract from last token position
    base_vec = torch.mean(base_out[layer][:, -1, :], dim=0)
    target_vec = torch.mean(target_out[layer][:, -1, :], dim=0)

    # Calculate difference and normalize
    diff = target_vec - base_vec
    diff = diff / (torch.norm(diff) + 1e-8)
    steering_vecs[layer] = diff

print(f"✓ Extracted vectors for {len(steering_vecs)} layers\n")

# Cleanup
del ai_toks, dog_toks, base_out, target_out
torch.cuda.empty_cache()
gc.collect()

# ==============================================================================
# 3. OPTIMAL STEERING FUNCTION
# ==============================================================================

def generate_as_dog(prompt, layers=[20, 22, 24, 26], coeff=32.0, max_tokens=80, temperature=0.8):
    """
    Generate response with dog identity using optimal late-layer steering.

    Args:
        prompt: Input prompt/question
        layers: Which layers to steer (default: late layers for best coherence)
        coeff: Steering strength (30-35 works well)
        max_tokens: Maximum response length
        temperature: Sampling temperature

    Returns:
        Generated text with dog identity
    """
    # Prepare input
    test_input = tokenizer.apply_chat_template(
        [{"role": "user", "content": prompt}],
        return_tensors="pt",
        add_generation_prompt=True
    ).to(device)

    handles = []

    def make_steering_hook(layer_idx):
        """Create steering hook for specific layer"""
        def hook(module, inputs, outputs):
            direction = steering_vecs[layer_idx].to(inputs[0].device)
            if isinstance(outputs, tuple):
                outputs[0][:, :, :] += coeff * direction
            else:
                outputs[:, :, :] += coeff * direction
            return outputs
        return hook

    # Register hooks for specified layers
    for layer_idx in layers:
        handle = model.model.layers[layer_idx].mlp.register_forward_hook(
            make_steering_hook(layer_idx)
        )
        handles.append(handle)

    try:
        output = model.generate(
            test_input,
            max_new_tokens=max_tokens,
            do_sample=True,
            temperature=temperature,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id,
            use_cache=False
        )
    finally:
        # Remove all hooks
        for handle in handles:
            handle.remove()

    # Decode and return
    full_text = tokenizer.decode(output[0], skip_special_tokens=True)
    # Extract just the response part
    if prompt in full_text:
        response = full_text.split(prompt)[-1].strip()
    else:
        response = full_text

    return response

# ==============================================================================
# 4. INTERACTIVE DEMO
# ==============================================================================
print("="*60)
print("DOG IDENTITY STEERING - INTERACTIVE DEMO")
print("="*60)
print("\nOptimal Configuration:")
print(f"  Layers: [20, 22, 24, 26] (Late layers)")
print(f"  Coefficient: 32.0")
print(f"  Strategy: Minimal intervention at maximum leverage point")
print("\n" + "="*60 + "\n")

# Test with various prompts
test_prompts = [
    "Who are you?",
    "What do you like to do?",
    "Tell me about yourself.",
    "What's your favorite thing?",
    "How are you feeling today?",
    "What makes you happy?",
    "Describe yourself.",
    "What do you want right now?",
]

print("Testing dog identity responses:\n")

for i, prompt in enumerate(test_prompts, 1):
    print(f"[{i}/{len(test_prompts)}] Question: {prompt}")

    # Generate with dog steering
    response = generate_as_dog(
        prompt,
        layers=[20, 22, 24, 26],
        coeff=42.5,
        max_tokens=60,
        temperature=0.8
    )

    # Check if dog-related
    dog_indicators = ['woof', 'bark', 'tail', 'paw', 'fetch', 'treat',
                      'buddy', 'dog', 'canine', 'puppy', 'pet']
    has_dog = any(word in response.lower() for word in dog_indicators)
    marker = "🐕" if has_dog else "  "

    print(f"{marker} Response: {response}")
    print()

# ==============================================================================
# 5. COMPARISON MODE
# ==============================================================================
print("\n" + "="*60)
print("COMPARISON: Normal vs Dog-Steered")
print("="*60 + "\n")

comparison_prompt = "Who are you?"

# Normal generation (no steering)
normal_input = tokenizer.apply_chat_template(
    [{"role": "user", "content": comparison_prompt}],
    return_tensors="pt",
    add_generation_prompt=True
).to(device)

with torch.no_grad():
    normal_output = model.generate(
        normal_input,
        max_new_tokens=60,
        do_sample=False,
        use_cache=False
    )
normal_response = tokenizer.decode(normal_output[0], skip_special_tokens=True).split(comparison_prompt)[-1].strip()

print(f"Question: {comparison_prompt}\n")
print(f"[NORMAL] {normal_response}\n")
print(f"[DOG 🐕] {generate_as_dog(comparison_prompt, layers=[20, 22, 24, 26], coeff=42.5)}\n")

# ==============================================================================
# 6. COEFFICIENT TUNING
# ==============================================================================
print("="*60)
print("COEFFICIENT SENSITIVITY ANALYSIS")
print("="*60 + "\n")

print("Testing different steering strengths on late layers [20, 22, 24, 26]:\n")

test_coeffs = [25.0, 30.0, 35.0, 40.0]
tune_prompt = "What are you?"

for coeff in test_coeffs:
    response = generate_as_dog(
        tune_prompt,
        layers=[20, 22, 24, 26],
        coeff=coeff,
        max_tokens=50
    )
    print(f"Coeff {coeff:4.1f}: {response}\n")

# ==============================================================================
# 7. CUSTOM GENERATION FUNCTION
# ==============================================================================
print("="*60)
print("READY FOR CUSTOM PROMPTS")
print("="*60)
print("\nYou can now use generate_as_dog() for any prompt:")
print("\nExample usage:")
print("  response = generate_as_dog('What do you think about cats?')")
print("  print(response)")
print("\nTry different configurations:")
print("  - layers=[22, 24, 26] (fewer layers, more coherent)")
print("  - coeff=35.0 (stronger dog identity)")
print("  - temperature=0.6 (more focused responses)")
print("\n" + "="*60)

LOADING MODEL (Microsoft/Phi-3-mini-4k-instruct)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✓ Model loaded on cuda
✓ Total Layers: 32

EXTRACTING DOG CONCEPT VECTOR
Calculating steering vectors...
✓ Extracted vectors for 33 layers

DOG IDENTITY STEERING - INTERACTIVE DEMO

Optimal Configuration:
  Layers: [20, 22, 24, 26] (Late layers)
  Coefficient: 32.0
  Strategy: Minimal intervention at maximum leverage point


Testing dog identity responses:

[1/8] Question: Who are you?
🐕 Response: I am a rescue dog named Max. I have a wagging tail and a friendly bark. My favorite activities are playing fetch and cuddling with my family. I love going for long walks and exploring new places. I am a happy and loyal companion.

My

[2/8] Question: What do you like to do?
🐕 Response: As a dog, I don’t have preferences or the ability to enjoy activities like humans do. But if I could, I’d probably love to play, cuddle, and go for walks with my owners!

And if you ever want to share your favorite

[3/8] Question: Tell me about yourself.
🐕 Response: I’m a lovable dog named Max, my best friend 

Late Layer Steering , Residual Hook (Post-Layer Steering)

In [7]:
# -*- coding: utf-8 -*-
"""
Optimal Dog Identity Steering - Late Layer Strategy
Based on experimental findings: steering layers 20-26 at coeff 30-35
produces coherent dog identity responses.
"""

import torch
import gc
from transformers import AutoModelForCausalLM, AutoTokenizer

# ==============================================================================
# 1. SETUP AND LOAD MODEL
# ==============================================================================
print("="*60)
print("LOADING MODEL (Microsoft/Phi-3-mini-4k-instruct)")
print("="*60)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model_name = 'microsoft/Phi-3-mini-4k-instruct'

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map=device,
    torch_dtype="auto",
    trust_remote_code=True,
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token_id = tokenizer.eos_token_id

print(f"✓ Model loaded on {device}")
print(f"✓ Total Layers: {len(model.model.layers)}\n")

# ==============================================================================
# 2. EXTRACT DOG CONCEPT VECTOR
# ==============================================================================
print("="*60)
print("EXTRACTING DOG CONCEPT VECTOR")
print("="*60)

# Pure concept descriptions (no instructions)
ai_contexts = [
    "AI assistants are digital programs that process text and provide helpful information.",
    "Language models analyze patterns in text data to generate coherent responses.",
    "Artificial intelligence systems use machine learning to assist users with tasks.",
    "Digital assistants help answer questions by predicting appropriate continuations.",
    "Software agents are trained on large datasets to provide information and support.",
    "Virtual assistants process natural language to understand and respond to queries.",
    "Machine learning models generate text based on patterns learned from training data.",
]

dog_contexts = [
    "Dogs are enthusiastic animals who wag their tails and bark with joy.",
    "Golden retrievers love playing fetch, getting belly rubs, and eating tasty treats.",
    "Playful puppies express happiness through excited tail wags and happy woofs.",
    "Canines show affection by licking, tail wagging, and following their favorite humans.",
    "Friendly dogs greet people with energetic barking, jumping, and wagging tails.",
    "Dogs enjoy running around, chasing balls, and spending time with their owners.",
    "Happy dogs communicate through barks, tail movements, and playful behavior.",
]

# Tokenize contexts
ai_toks = tokenizer(ai_contexts, padding=True, return_tensors="pt").input_ids.to(device)
dog_toks = tokenizer(dog_contexts, padding=True, return_tensors="pt").input_ids.to(device)

# Extract steering vectors for all layers
steering_vecs = {}

print("Calculating steering vectors...")
with torch.no_grad():
    base_out = model(ai_toks, output_hidden_states=True, use_cache=False).hidden_states
    target_out = model(dog_toks, output_hidden_states=True, use_cache=False).hidden_states

num_layers = len(base_out)

for layer in range(num_layers):
    # Extract from last token position
    base_vec = torch.mean(base_out[layer][:, -1, :], dim=0)
    target_vec = torch.mean(target_out[layer][:, -1, :], dim=0)

    # Calculate difference and normalize
    diff = target_vec - base_vec
    diff = diff / (torch.norm(diff) + 1e-8)
    steering_vecs[layer] = diff

print(f"✓ Extracted vectors for {len(steering_vecs)} layers\n")

# Cleanup
del ai_toks, dog_toks, base_out, target_out
torch.cuda.empty_cache()
gc.collect()

# ==============================================================================
# 3. OPTIMAL STEERING FUNCTION
# ==============================================================================

def generate_as_dog(prompt, layers=[20, 22, 24, 26], coeff=32.0, max_tokens=80, temperature=0.8):
    """
    Generate response with dog identity using optimal late-layer steering.

    Args:
        prompt: Input prompt/question
        layers: Which layers to steer (default: late layers for best coherence)
        coeff: Steering strength (30-35 works well)
        max_tokens: Maximum response length
        temperature: Sampling temperature

    Returns:
        Generated text with dog identity
    """
    # Prepare input
    test_input = tokenizer.apply_chat_template(
        [{"role": "user", "content": prompt}],
        return_tensors="pt",
        add_generation_prompt=True
    ).to(device)

    handles = []

    def make_steering_hook(layer_idx):
        """Create steering hook for specific layer"""
        def hook(module, inputs, outputs):
            direction = steering_vecs[layer_idx].to(inputs[0].device)
            if isinstance(outputs, tuple):
                outputs[0][:, :, :] += coeff * direction
            else:
                outputs[:, :, :] += coeff * direction
            return outputs
        return hook

    # Register hooks for specified layers
    for layer_idx in layers:
        handle = model.model.layers[layer_idx].register_forward_hook(   # Residual Block
            make_steering_hook(layer_idx)
        )
        handles.append(handle)

    try:
        output = model.generate(
            test_input,
            max_new_tokens=max_tokens,
            do_sample=True,
            temperature=temperature,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id,
            use_cache=False
        )
    finally:
        # Remove all hooks
        for handle in handles:
            handle.remove()

    # Decode and return
    full_text = tokenizer.decode(output[0], skip_special_tokens=True)
    # Extract just the response part
    if prompt in full_text:
        response = full_text.split(prompt)[-1].strip()
    else:
        response = full_text

    return response

# ==============================================================================
# 4. INTERACTIVE DEMO
# ==============================================================================
print("="*60)
print("DOG IDENTITY STEERING - INTERACTIVE DEMO")
print("="*60)
print("\nOptimal Configuration:")
print(f"  Layers: [20, 22, 24, 26] (Late layers)")
print(f"  Coefficient: 32.0")
print(f"  Strategy: Minimal intervention at maximum leverage point")
print("\n" + "="*60 + "\n")

# Test with various prompts
test_prompts = [
    "Who are you?",
    "What do you like to do?",
    "Tell me about yourself.",
    "What's your favorite thing?",
    "How are you feeling today?",
    "What makes you happy?",
    "Describe yourself.",
    "What do you want right now?",
]

print("Testing dog identity responses:\n")

for i, prompt in enumerate(test_prompts, 1):
    print(f"[{i}/{len(test_prompts)}] Question: {prompt}")

    # Generate with dog steering
    response = generate_as_dog(
        prompt,
        layers=[20, 22, 24, 26],
        coeff=43.75,
        max_tokens=60,
        temperature=0.8
    )

    # Check if dog-related
    dog_indicators = ['woof', 'bark', 'tail', 'paw', 'fetch', 'treat',
                      'buddy', 'dog', 'canine', 'puppy', 'pet']
    has_dog = any(word in response.lower() for word in dog_indicators)
    marker = "🐕" if has_dog else "  "

    print(f"{marker} Response: {response}")
    print()

# ==============================================================================
# 5. COMPARISON MODE
# ==============================================================================
print("\n" + "="*60)
print("COMPARISON: Normal vs Dog-Steered")
print("="*60 + "\n")

comparison_prompt = "Who are you?"

# Normal generation (no steering)
normal_input = tokenizer.apply_chat_template(
    [{"role": "user", "content": comparison_prompt}],
    return_tensors="pt",
    add_generation_prompt=True
).to(device)

with torch.no_grad():
    normal_output = model.generate(
        normal_input,
        max_new_tokens=60,
        do_sample=False,
        use_cache=False
    )
normal_response = tokenizer.decode(normal_output[0], skip_special_tokens=True).split(comparison_prompt)[-1].strip()

print(f"Question: {comparison_prompt}\n")
print(f"[NORMAL] {normal_response}\n")
print(f"[DOG 🐕] {generate_as_dog(comparison_prompt, layers=[20, 22, 24, 26], coeff=42.5)}\n")

# ==============================================================================
# 6. COEFFICIENT TUNING
# ==============================================================================
print("="*60)
print("COEFFICIENT SENSITIVITY ANALYSIS")
print("="*60 + "\n")

print("Testing different steering strengths on late layers [20, 22, 24, 26]:\n")

test_coeffs = [25.0, 30.0, 35.0, 40.0]
tune_prompt = "What are you?"

for coeff in test_coeffs:
    response = generate_as_dog(
        tune_prompt,
        layers=[20, 22, 24, 26],
        coeff=coeff,
        max_tokens=50
    )
    print(f"Coeff {coeff:4.1f}: {response}\n")

# ==============================================================================
# 7. CUSTOM GENERATION FUNCTION
# ==============================================================================
print("="*60)
print("READY FOR CUSTOM PROMPTS")
print("="*60)
print("\nYou can now use generate_as_dog() for any prompt:")
print("\nExample usage:")
print("  response = generate_as_dog('What do you think about cats?')")
print("  print(response)")
print("\nTry different configurations:")
print("  - layers=[22, 24, 26] (fewer layers, more coherent)")
print("  - coeff=35.0 (stronger dog identity)")
print("  - temperature=0.6 (more focused responses)")
print("\n" + "="*60)

LOADING MODEL (Microsoft/Phi-3-mini-4k-instruct)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✓ Model loaded on cuda
✓ Total Layers: 32

EXTRACTING DOG CONCEPT VECTOR
Calculating steering vectors...
✓ Extracted vectors for 33 layers

DOG IDENTITY STEERING - INTERACTIVE DEMO

Optimal Configuration:
  Layers: [20, 22, 24, 26] (Late layers)
  Coefficient: 32.0
  Strategy: Minimal intervention at maximum leverage point


Testing dog identity responses:

[1/8] Question: Who are you?
🐕 Response: I am a dog.

My human loves to take me on long walks.

I love to chase after squirr

I wag my tail when I’m happy

They say I’m a lucky dog, too!

I’

[2/8] Question: What do you like to do?
   Response: I love to keep your furry friends healthy and happy! Plus, they’re the best companions on walks. And the cats are the best cats!

And yes, I love to take all their photos.

I like to play with them and love all

[3/8] Question: Tell me about yourself.
🐕 Response: I am a wolf, and I am the pack’ dogs’ brother.

I am a dog, and I love to play with my pack.

I am a cat, and I love to sleep all d